In [1]:
from arch.bootstrap import StationaryBootstrap
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm


In [2]:
sp_tech = pd.read_excel("C:/Users/limam/Documents/GitHub/Datathon_2023/Dados/sinais_ind_tecnicos.xlsx")


In [3]:
sp_tech

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12
0,2015-03-02,274.354004,242425100,1,1,1,1,1,0,1,0,1,1,1,1,1,1
1,2015-03-09,286.393005,259502000,1,1,1,1,1,1,1,0,1,1,1,1,1,1
2,2015-03-16,267.959991,206778200,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,2015-03-23,242.712997,179497700,0,0,1,1,1,1,1,0,1,1,1,1,1,1
4,2015-03-30,260.597992,150120700,1,1,0,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,2023-10-02,27935.089844,82183775698,1,1,1,1,1,0,0,0,0,0,0,0,0,0
449,2023-10-09,27159.652344,72673654487,1,0,1,1,1,1,1,0,0,0,0,0,0,0
450,2023-10-16,29993.896484,113402383094,1,1,1,1,1,1,1,1,0,0,0,0,0,0
451,2023-10-23,33086.234375,38363572311,1,1,1,1,1,1,1,1,1,0,1,0,0,0


In [ ]:
Y

In [11]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats
from sklearn.utils import resample

# Read the Excel file into a pandas DataFrame
sp_tech = pd.read_excel("C:/Users/limam/Documents/GitHub/Datathon_2023/Dados/sinais_ind_tecnicos.xlsx")

# Assuming 'premium' is already a column in sp_tech DataFrame
t = len(sp_tech)
n = 14  # number of technical indicators
resultados = np.zeros((n, 4))  # To store coefficient, t-stat, p-value, and R^2
residuos = np.zeros((t - 1, n))  # t-1 because we're starting from the second element
j = 3  # Column index where technical indicators data starts

sp_tech['return'] = sp_tech['Adj Close'].pct_change()


# Loop for regression
for i in range(n):
    Y = sp_tech['return'][2:t].values
    X = sp_tech.iloc[1:(t-1), j].values
    X = sm.add_constant(X)  # Add the intercept
    model = sm.OLS(Y, X).fit()    
    # Bootstrapping
    coef_bootstraps = []
    
    for _ in range(1000):
        # Resample with replacement
        index_resampled = resample(np.arange(len(X)), replace=True)
        X_resampled = X[index_resampled]
        Y_resampled = Y[index_resampled]
        model_resampled = sm.OLS(Y_resampled, X_resampled).fit()
        coef_bootstraps.append(model_resampled.params[1])  # Get the coefficient for the indicator

    coef_bootstraps = np.array(coef_bootstraps)
    std_error = coef_bootstraps.std(ddof=1)
    
    # Calculate t-statistic and p-value
    coef = model.params[1]
    t_stat = coef / std_error
    p_value = 2 * (1 - stats.t.cdf(np.abs(t_stat), df=n-2))  # Two-tailed test
    
    # Get R-squared from the model
    r_squared = model.rsquared
    
    # Store results in resultados array
    resultados[i, :] = coef, t_stat, p_value, r_squared
    
    # Store residuals in residuos array
    #residuos = model.resid

    # Move to the next indicator
    j += 1

# Create a DataFrame to display the results nicely
results_df = pd.DataFrame(resultados, columns=['Coefficient', 'T-Statistic', 'P-Value', 'R-Squared'])

print(results_df)


    Coefficient  T-Statistic   P-Value  R-Squared
0      0.018293     1.969769  0.072387   0.008158
1      0.019564     2.061403  0.061618   0.009331
2      0.016353     1.753798  0.104944   0.006511
3      0.020147     2.162448  0.051486   0.009854
4      0.018410     2.022062  0.066045   0.008262
5      0.018368     2.044308  0.063506   0.008191
6      0.018035     1.936181  0.076752   0.007738
7      0.018767     2.064953  0.061232   0.008442
8      0.015792     1.755371  0.104666   0.006036
9      0.013484     1.426770  0.179144   0.004350
10     0.012171     1.313835  0.213465   0.003591
11     0.009888     1.071736  0.304920   0.002326
12     0.009916     1.100923  0.292524   0.002397
13     0.011090     1.165185  0.266584   0.002953


In [10]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Read the Excel file into a pandas DataFrame
sp_tech = pd.read_excel("C:/Users/limam/Documents/GitHub/Datathon_2023/Dados/sinais_ind_tecnicos.xlsx")

# Handle NaN values for 'Adj Close', perhaps with forward fill or dropna:
sp_tech['Adj Close'].fillna(method='ffill', inplace=True)

# Calculate the percentage returns of the 'Adj Close' column
sp_tech['return'] = sp_tech['Adj Close'].pct_change().dropna()

# Initialization
n = 14  # Number of technical indicators
t = len(sp_tech)
resultados = np.zeros((n, 4))  # To store coefficient, t-statistic, R^2, and p-value
residuos = np.zeros((t - 2, n))  # t-2 because the first return will be NaN
summaries = []  # To store summary objects for later use
j = 3  # Column index where technical indicators data starts

# Loop for regression
for i in range(n):
    # The dependent variable (Y) and independent variable (X)
    Y = sp_tech['return'][1:].values  # Use values after the first NaN
    X = sp_tech.iloc[:-1, j+i].values  # Use values up to the last to match with Y
    X = sm.add_constant(X)  # Add the intercept
    
    # Perform regression
    model = sm.OLS(Y, X).fit()
    summaries.append(model.summary())
    
    # Store results
    resultados[i] = [model.params[1], model.tvalues[1], model.rsquared, model.pvalues[1]]
    #residuos = model.resid
    
    # Print out each summary if needed
    print(summaries[i])

# Now, 'resultados' contains the regression results for each technical indicator
# and 'residuos' contains the residuals for each regression.


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     3.739
Date:                Mon, 06 Nov 2023   Prob (F-statistic):             0.0538
Time:                        09:13:59   Log-Likelihood:                 400.79
No. Observations:                 452   AIC:                            -797.6
Df Residuals:                     450   BIC:                            -789.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0051      0.007      0.716      0.4